In [11]:
"""
GravMag: 3D forward modeling of total-field magnetic anomaly using prolate
ellipsoids (model with isotropic and anisotropic susceptibilities)
"""
from fatiando import utils, gridder
from fatiando.gravmag import prolate_ellipsoid
from fatiando.mesher import ProlateEllipsoid
from fatiando.vis import mpl

# The regional field
F, inc, dec = 23500., 30, -15

bounds = [-5000, 5000, -5000, 5000, 0, 5000]
# Create a model formed by two ellipsoids
# The first ellipsoid does not have remanent magnetization and
# has an anisotropic susceptibility (k1 = 0.2, k2 = 0.15, k3 = 0.12). 
# The second has a remanent magnetization of 2 A/m
# and an isotropic susceptibility of 0.01
model = [ProlateEllipsoid(0., -2000., 1000., 4000., 500., -29., 0., 0.,
                          {'k': [0.2, 0.15, 0.12, 0., 24., 90.]}),
         ProlateEllipsoid(2500., 2500., 1000., 900., 800., 90., -32., 8.,
                          {'remanence': [2, 90., 0.],
                           'k': [0.01, 0.01, 0.01, 34., 18., 87.]})]

# Create a regular grid at 0m height
shape = (200, 200)
area = bounds[:4]
xp, yp, zp = gridder.regular(area, shape, z = 0)

# Calculate the total-field anomaly
tf = prolate_ellipsoid.tf(xp, yp, zp, model, F, inc, dec)

# Plot
mpl.figure()
mpl.title("Total-field anomaly (nT)")
mpl.axis('scaled')
mpl.contourf(yp, xp, tf, shape, 15)
mpl.colorbar()
mpl.xlabel('East y (km)')
mpl.ylabel('North x (km)')
mpl.m2km()
mpl.show()